# Coeficiente de Tanimoto

$T _{c}(A,B) = \frac{c}{a+b-c}$

- a: número de características presentes na molécula A
- b: número de características presentes na molécula B
- c: número de características compartilhadas pelas moléculas A e B

# Coeficiente de Dice

$D_{c}(A,B) = \frac{c}{\frac{1}{2}(a+b)}$

- a: número de características presentes na molécula A
- b: número de características presentes na molécula B
- c: número de características compartilhadas pelas moléculas A e B


In [1]:
#-----Base-----#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#-----RDKit-----#
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit import DataStructs
from rdkit.DataStructs import ExplicitBitVect
from rdkit.ML.Cluster import Butina

In [2]:
class Clusterizacao:

    def __init__(self, dataframe: pd.DataFrame, col_fp: str) -> None:
        '''
            Input:
                - dataframe: pd.DataFrame contendo a representação fingerprint
                - col_fp: nome da coluna contendo a representação fingerprint
        '''

        self.dataframe = dataframe.copy()
        self.col_fp = col_fp

    def clusters(self, algoritmo: str='tanimoto', cutoff: float=0.8) -> pd.DataFrame:
        '''
            Input:
                - algoritmo: algoritmo para cálculo de similaridade
                - cutoff: valor de corte de SIMILARIDADE para clusterização. 
                    **ATENÇÃO:** Não inserir o valor de DISTÂNCIA como cutoff.
        '''

        df = self.butina_cluster(similaridade=algoritmo, cutoff=cutoff)

        return df
    
    def matrix_Tanimoto(self, col_frame: str='ROMol') -> pd.DataFrame:
        '''
            Observação: Para fins puramente de visualização.

            Descrição:
                Cria uma DataFrame (matrix) de similaridade usando o cálculo de Tanimoto.
        
            Input:
                - col_frame: nome da coluna contendo a representação em frame das moléculas (para representar as linhas e colunas da matrix)
        '''
    
        # Obtem os fingerprints (ExplicitBitVect)
        bitVect_array = self.dataframe[self.col_fp].to_list()
    
        # Gera uma matriz len(dataframe) x len(dataframe) de np.nan
        matrix = []
        
        for i in self.dataframe.index:
    
            array_similaridade = DataStructs.BulkTanimotoSimilarity(bitVect_array[i], bitVect_array)
            matrix.append(array_similaridade)
    
        return pd.DataFrame(data=matrix, index=self.dataframe[col_frame], columns=self.dataframe[col_frame])
    
    def matrix_Dice(self, col_frame: str='ROMol') -> pd.DataFrame:
        '''
            Observação: Para fins puramente de visualização.

            Descrição:
                Cria uma DataFrame (matrix) de similaridade usando o cálculo de Tanimoto.
        
            Input:
                - col_frame: nome da coluna contendo a representação em frame das moléculas (para representar as linhas e colunas da matrix)
        '''
    
        # Obtem os fingerprints (ExplicitBitVect)
        bitVect_array = self.dataframe[self.col_fp].to_list()
    
        # Gera uma matriz len(dataframe) x len(dataframe) de np.nan
        matrix = []
        
        for i in self.dataframe.index:
    
            array_similaridade = DataStructs.BulkDiceSimilarity(bitVect_array[i], bitVect_array)
            matrix.append(array_similaridade)
    
        return pd.DataFrame(data=matrix, index=self.dataframe[col_frame], columns=self.dataframe[col_frame])
    
    def butina_cluster(self, similaridade: str='tanimoto', cutoff: float=0.8) -> pd.DataFrame:
        '''
            Descrição:
                Clusteriza os registros da DataFrame baseado no algoritmo de Butina.
        
            Input:
                - similaridade: algoritmo de similaridade aplicado: 
                    - 'tanimoto': BulkTanimotoSimilarity()
                    - 'dice': BulkDiceSimilarity()
                - cutoff: valor de corte de SIMILARIDADE para clusterização.
        '''

        # Calcula a distância de similaridade
        cutoff = 1 - cutoff
        
        # Obtem os fingerprints (ExplicitBitVect)
        bitVect_array = self.dataframe[self.col_fp].to_list()

        # Calcula a distância de similaridade (1 - similaridade)
        matrix_distancia = []
        
        if similaridade == 'dice':
            # Comparar todos os pares de fingerprints
            for i in range(len(bitVect_array)):
                for j in range(i):
                    # Calcular similaridade de Dice
                    similarity = DataStructs.DiceSimilarity(bitVect_array[i], bitVect_array[j])
                    matrix_distancia.append(1 - similarity)
        else:
            # Comparar todos os pares de fingerprints
            for i in range(len(bitVect_array)):
                for j in range(i):
                    # Calcular similaridade de Tanimoto
                    similarity = DataStructs.TanimotoSimilarity(bitVect_array[i], bitVect_array[j])
                    matrix_distancia.append(1 - similarity)

        # Clusteriza os registros
        clusters = Butina.ClusterData(data=matrix_distancia, nPts=len(bitVect_array), distThresh=cutoff, isDistData=True)
        clusters = sorted(clusters, key=len, reverse=True)

        # Insere na DataFrame o cluster de cada registro
        self.dataframe['ButinaCluster'] = np.nan
        
        for indice in range(len(clusters)):
            # Obtem o índice de cada registro dentro do cluster
            for registro in clusters[indice]:
                self.dataframe.loc[registro, 'ButinaCluster'] = indice

        return self.dataframe